In [11]:
%load_ext autoreload
%autoreload 2
from datetime import datetime

from deltas.pipeline import data, classifier, evaluation
from deltas.model import base, downsample
import deltas.plotting.plots as plots
from tqdm import tqdm

import numpy as np
import pandas as pd
# np.random.seed(10)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:

N1 = 100
N2 = 10
m = 1
v = 1
costs = (1, 1)  # change for (1, 10) to increase results
# Gaussian (not always seperable)
dfs = []
len_required = 100
for i in tqdm(range(len_required*10)):
    data_clf = data.get_data(
        m1=[-m, -m],
        m2=[m, m],
        cov1=[[v, 0], [0, v]],
        cov2=[[v, 0], [0, v]],
        N1=N1,
        N2=N2,
        scale=False,
        test_nums=[10000, 10000],
    )

    model = 'MLP'
    model = 'MLP-Gaussian'

    classifiers_dict = classifier.get_classifier(
        data_clf=data_clf,
        model=model,
        _plot=False)
    data_clf['clf'] = classifiers_dict['Baseline']
    X = data_clf['data']['X']
    y = data_clf['data']['y']
    clf = data_clf['clf']
    # deltas_model = downsample.downsample_deltas(
    #     clf).fit(X, y, _print=True, _plot=True, max_trials=10000)
    # deltas_model = base.base_deltas(
    #     clf).fit(X, y, grid_search=True, _print=True, _plot=True)
    deltas_model = downsample.downsample_deltas(clf).fit(X, y,
                                                         alpha=1,
                                                         _print=False,
                                                         _plot=False,
                                                         method='supports-prop-update_mean',
                                                         max_trials=10000,
                                                         parallel=True)

    if deltas_model.is_fit == True:
        classifiers_dict['Our Method'] = deltas_model
        scores_df = evaluation.eval_test(classifiers_dict,
                                         data_clf['data_test'], _print=False, _plot=False)
        dfs.append(scores_df)
    else:
        print('not fit deltas')
    if len(dfs) == len_required:
        break

  0%|          | 0/1000 [00:00<?, ?it/s]

/home/matt/anaconda3/envs/deltas/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/matt/anaconda3/envs/deltas/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
  0%|          | 1/1000 [00:01<27:38,  1.66s/it]/home/matt/anaconda3/envs/deltas/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/matt/anaconda3/envs/deltas/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the 

In [13]:
print(len(dfs))

100


In [14]:
df = pd.concat(dfs, axis=0)
mean = {}
std = {}
index = df.index.unique().to_list()
cols = df.columns.to_list()
for method in index:
    mean[method] = df.loc[method].mean().to_list()
    std[method] = df.loc[method].std().to_list()

mean_df = pd.DataFrame.from_dict(mean, orient='index', columns=cols)
std_df = pd.DataFrame.from_dict(std, orient='index', columns=cols)

m = mean_df.to_dict('list')
s = std_df.to_dict('list')
metrics = mean_df.columns.to_list()
methods = mean_df.index.to_list()
sf = 5
for metric in metrics:
    means = m[metric]
    sts = s[metric]
    mx = np.argmax(means)
    for i in range(len(means)):
        m_str = str(means[i])[1:sf]
        if i == mx:
            m_str = f"\\textbf{{{m_str}}}"
        s_str = str(sts[i])[1:sf-1]
        m[metric][i] = f'${m_str} \\pm {s_str}$'

method_map = {
    'Baseline': 'Baseline',
    'SMOTE': "SMOTE \cite{Chawla_2002_JAIR}",
    'Balanced Weights': 'BW',
    'BMR': 'BMR \cite{Bahnsen_2014_SIAM}',
    'Threshold': 'Thresh \cite{Sheng_2006_AAAI}',
    'Our Method': 'Our Method',
}
meths_new = []
for me in methods:
    meths_new.append(method_map[me])
m['Methods'] = meths_new
df = pd.DataFrame(m)  # .set_index('Methods')
meths = df.pop('Methods')
df.insert(0, 'Methods', meths)
latex_str = df.to_latex(index=False)

# print(f'\n% Gaussian - {len_required}')
# print('\\begin{tabular}{@{}lrrrr@{}}'+latex_str[22:])
print(mean_df)

                  Accuracy    G-Mean   ROC-AUC        F1
Baseline          0.597467  0.431783  0.974515  0.319772
SMOTE             0.907583  0.906466  0.974086  0.903307
Balanced Weights  0.909358  0.908477  0.973895  0.905684
BMR               0.889364  0.884692  0.974515  0.880554
Threshold         0.890963  0.886660  0.974515  0.883056
Our Method        0.910071  0.909075  0.974515  0.908826


In [15]:
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
with open(f"Results-0-Gaussian.txt", "w") as text_file:
    text_file.write(f'% {dt_string}\n')
    text_file.write(f'% Gaussian - {len_required} runs\n')
    text_file.write('\\begin{tabular}{@{}lrrrr@{}}'+latex_str[22:])